In [55]:
%pip install --quiet -U  langchain langchain-google-genai  langgraph

In [56]:
from google.colab import userdata
google_api_key = userdata.get('GEMINI_API_KEY')


In [57]:
from langchain_google_genai import ChatGoogleGenerativeAI
# Initialize an instance of the ChatGoogleGenerativeAI with specific parameters
llm: ChatGoogleGenerativeAI = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",  # Specify the model to use
    api_key=google_api_key,     # Provide the Google API key for authentication
)

In [58]:
from typing_extensions import TypedDict
from typing import List

class TextAnalysisState(TypedDict):
  text : str
  classification : str
  entities : List[str]
  summary : str



2. Nodes
  * classification
  * entities
  * summary

# "NODE 1"     Text_Classify

In [59]:

# node 1    classification

def text_classify(state: TextAnalysisState) -> TextAnalysisState:
  print('--classification--', state)
  text : str = state['text']
  prompt : str = f"""
      Text will be given as input
      Analyze the text and categorize it into one of the following domains:
      NEWS, BLOG, RESEARCH, PRODUCT REVIEWS, ACADEMIC ESSAYS, PRESENTATIONS, or EMAIL COMMUNICATIONS.
      Return the classification type.
      Text: {text}
"""

  output = llm.invoke(prompt.format(text=text))
  state['classification'] = output.content.strip()
  return state




# Node 2 -->

In [60]:
# node 2    Entities Extraction

def entities_extraction(state : TextAnalysisState) -> TextAnalysisState:
  print('--Classification--', state)
  text : str = state['text']
  classification : str = state['classification']

  prompt : str = f"""
  You will be given Text as input.
  Extract entities present in text i.e names, locations, organizations, dates, and any significant terms that contribute to the main theme.
  Your job is to just return the entities present in json i.e entities  {{"entities": ["zia khan lives in karachi"]}}  entity1 : zia khan , entity2 : karachi

  Input Text is : {text}
  Classification is: {classification}
  Output JSON Output:
  {"entities": ["1"...]}
  """

  output = llm.invoke(prompt.format(text=text, classification = classification))
  return {'classification': output.content }


In [34]:
# sample_state: TextAnalysisState = {
#     'text': input_text,
#     'classification': 'NEWS'
# }

# # text_classify(sample_state)  # Call text_classify to add classification to sample_state
# enti_output = entities_extraction(sample_state)
# print(enti_output)

NameError: name 'input_text' is not defined

# "NODE3"  text_summary

In [61]:
# node 3 text summary

def text_summary(state: TextAnalysisState) -> TextAnalysisState:
  print('--summary--', state)
  text: str = state['text']
  classification: str = state['classification']
  entities : List[str] = state['entities']

  prompt: str = f"""
  You will be given text and its classification as input.
  Summarize the given text within 150 words, capturing the key information and main points.
  Input Text: {text}
  Classification: {classification}
  Entities is : {entities}
  Summary:
  """
  output = llm.invoke(prompt)
  state['summary'] = output.content.strip()
  return state



In [ ]:
# from your_grok_library import GrokAI  # Replace with actual library import

# # Initialize an instance of Grok AI
# grok_ai = GrokAI(
#     api_key=your_grok_api_key,  # Provide your Grok API key
#     model="grok_model_name"      # Specify the model to use if applicable
# )

In [62]:
from langgraph.graph import StateGraph, START,END
from langgraph.graph.state import CompiledStateGraph

In [70]:
# Define the state graph
text_analysis_builder = StateGraph(TextAnalysisState)

# Add nodes to the graph with unique names
text_analysis_builder.add_node("classification_node", text_classify)
text_analysis_builder.add_node("entity_extraction_node", entities_extraction)
text_analysis_builder.add_node("summarization_node", text_summary)

# Define graph edges with updated node names
text_analysis_builder.add_edge(START, "classification_node")
text_analysis_builder.add_edge("classification_node", "entity_extraction_node")
text_analysis_builder.add_edge("entity_extraction_node", "summarization_node")
text_analysis_builder.add_edge("summarization_node", END)

# Compile the graph (only once)
text_analysis_compiledgraph = text_analysis_builder.compile()

# Take Input from the User
input_text = input("Enter text for analysis: ")

# Define the initial state with user input
state = {
    'text': input_text,
    'classification': '',
    'entities': [],
    'summary': ''
}

# Run the state graph and get the results
result = text_analysis_compiledgraph


# Display the results
print("\n--- Text Analysis Results ---")
print(f"Classification: {result['classification']}")
print(f"Entities: {result['entities']}")
print(f"Summary: {result['summary']}")

Enter text for analysis: artificial intelligence (AI) model that uses deep learning to perform natural language processing (NLP) tasks. LLMs are trained on large amounts of data, such as text from the internet or Wikipedia, to learn statistical relationships and understand how words and phrases relate to each other. They can then be used to generate or translate text, answer questions, summarize documents, and more. LLMs are also known as neural networks (NNs) because they are computing systems inspired by the human brain.

--- Text Analysis Results ---


TypeError: 'CompiledStateGraph' object is not subscriptable

In [50]:


# def text_classify(state: TextAnalysisState) -> TextAnalysisState:
#     text : input_text = state['text'].lower()  # Convert to lowercase once

#     # Classify based on keywords in the text
#     if "breaking news" in text or "report" in text or "headline" in text:
#         state['classification'] = 'News'
#     elif "blog" in text or "opinion" in text or "personal" in text:
#         state['classification'] = 'Blog'
#     elif "research" in text or "study" in text or "paper" in text:
#         state['classification'] = 'Research'
#     else:
#         state['classification'] = 'Other'

#     print('--classification--', state['classification'])


AttributeError: 'CompiledStateGraph' object has no attribute 'execute'

In [42]:
#  sample_state variable for TextAnalysisState

# sample_state: TextAnalysisState = {
#    "text": "Education in Pakistan is overseen by the Federal Ministry of Education and the provincial governments, while the federal government mostly assists in curriculum development, accreditation and the financing of research and development. 'The State shall provide free and to all children of the age of five to sixteen years in such a manner as may be determined by law'.Considering the challenges confronting the youth in Pakistan, it's hardly astonishing that well-off and educated young individuals and professionals are opting to depart the country whenever they have the opportunity, exacerbating the brain drain phenomenon."
# }
